In [1]:
from pydantic import BaseModel

class MemberBase(BaseModel):
    id:int
    name:str
    grade:str

import sqlite3
def create_connection():
    conn = sqlite3.connect("member.db")
    return conn

def create_table():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("""
                    CREATE TABLE IF NOT EXISTs member(
                   id INTEGER PRIMARY KEY,
                   name TEXT NOT NULL,
                   grade TEXT NOT NULL
                    )
""")
    conn.commit()
    conn.close()

create_table()
def register(member:MemberBase):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("insert into member(id,name,grade) values(?,?,?)",(member.id,member.name,member.grade))
    conn.commit()
    conn.close()

def getmember():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("Select * from member")
    data = cursor.fetchall()
    conn.commit()
    conn.close()
    return data

def delete_member(member_id: int):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("DELETE FROM member WHERE id=?", (member_id,))
    conn.commit()
    conn.close()

def update_member(member_id: int, new_data: MemberBase):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("UPDATE member SET name=?, grade=? WHERE id=", (new_data.name, new_data.grade, member_id))
    conn.commit()
    conn.close()

def find_member_by_id(member_id: int):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM member WHERE id=?", (member_id,))
    member_data = cursor.fetchone()
    conn.close()
    
    if member_data:
        member = MemberBase(id=member_data[0], name=member_data[1], grade=member_data[2])
        return member
    return None

In [2]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse

In [ ]:
app = FastAPI()
@app.post("/member/")
def create_member_endpoint(member:MemberBase):
    id = register(member)
    return{"id":id,**member.model_dump()}

@app.get("/member")
def get_all():
    return getmember()

@app.delete("/member/{id}")
def delete_mem(id: int):
    delete_member(id)
    return getmember()

@app.get("/member/{id}")
def find_member_by_id(member_id: int):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM member WHERE id=?", (member_id,))
    member_data = cursor.fetchone()
    conn.close()
    if member_data:
        return {"id": member_data[0], "name": member_data[1], "grade": member_data[2]}
    return None

@app.put("/member/{id}")
def update_member(id: int, member: MemberBase):
    update_member(id, member)
    return getmember()

In [5]:
import asyncio
import uvicorn

if __name__ == "__main__" :
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [15540]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53846 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53847 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53847 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:53852 - "GET /member HTTP/1.1" 200 OK
INFO:     127.0.0.1:53863 - "POST /member/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:53863 - "POST /member/ HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\BOM\anaconda3\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\BOM\anaconda3\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\BOM\anaconda3\lib\site-packages\fastapi\applications.py", line 292, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\BOM\anaconda3\lib\site-packages\starlette\applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\BOM\anaconda3\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\Users\BOM\anaconda3\lib\site-packages\starlette\middleware\errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "c:\Users\BOM\anaconda3\lib\sit

INFO:     127.0.0.1:53867 - "POST /member/ HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\BOM\anaconda3\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\BOM\anaconda3\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\BOM\anaconda3\lib\site-packages\fastapi\applications.py", line 292, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\BOM\anaconda3\lib\site-packages\starlette\applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\BOM\anaconda3\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\Users\BOM\anaconda3\lib\site-packages\starlette\middleware\errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "c:\Users\BOM\anaconda3\lib\sit